Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [85]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [110]:
n_male = df[df["gender"]=="Male"]["gender"].count()
n_all = df.shape[0]
n_female = n_all - n_male
aspect = n_female / n_male
print(f'Количество мужчин: {n_male}')
print(f'Количество женщин: {n_female}')
print(f'Соотношение женщин/мужчин: {round(aspect, 2)}')

Количество мужчин: 3555
Количество женщин: 3488
Соотношение женщин/мужчин: 0.98


##### 2. Какое количество уникальных значений у поля InternetService?

In [87]:
len(df["InternetService"].unique())

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [88]:
total_charges_numeric = df["TotalCharges"].copy()
for i in range(total_charges_numeric.shape[0]):
    try:
        pd.to_numeric(total_charges_numeric.iloc[i])
    except ValueError:
        total_charges_numeric.iloc[i] = np.nan
total_charges_numeric = total_charges_numeric.astype(float)
total_charges_numeric.dropna(inplace=True)

print(f'Median: {total_charges_numeric.median()}')
print(f'Mean: {total_charges_numeric.mean()}')
print(f'Std: {total_charges_numeric.std()}')

Median: 1397.475
Mean: 2283.300440841866
Std: 2266.771361883145


В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [91]:
df.loc[df['PhoneService'] == 'Yes', 'PhoneService'] = 1
df.loc[df['PhoneService'] == 'No', 'PhoneService'] = 0
df['PhoneService']

0       0
1       1
2       1
3       0
4       1
       ..
7038    1
7039    1
7040    0
7041    1
7042    1
Name: PhoneService, Length: 7043, dtype: object

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [68]:
total_charges_numeric = df["TotalCharges"].copy()
for i in range(total_charges_numeric.shape[0]):
    try:
        pd.to_numeric(total_charges_numeric.iloc[i])
    except ValueError:
        total_charges_numeric.iloc[i] = np.nan
total_charges_numeric = total_charges_numeric.astype(np.float32)
total_charges_numeric.fillna(0, inplace=True)

print(f'Median: {total_charges_numeric.median()}')
print(f'Mean: {total_charges_numeric.mean()}')
print(f'Std: {total_charges_numeric.std()}')

df["TotalCharges"] = total_charges_numeric

Median: 1.0
Mean: 0.9031662940979004
Std: 0.29575222730636597


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [92]:
df.loc[df['Churn'] == 'Yes', 'Churn'] = 1
df.loc[df['Churn'] == 'No', 'Churn'] = 0
df["Churn"]

0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7043, dtype: object

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [108]:
def to_binary(series):
    col = series.copy()
    col[col == 'Yes'] = 1
    col[col == 'No'] = 0
    col[col == 'No internet service'] = 0 
    return col


def feature_to_binary(df, cols):
    for col in cols:
        df[col] = to_binary(df[col])
    

feature_to_binary(df, ['StreamingMovies', 'StreamingTV', 'TechSupport'])

print(df['TechSupport'].unique())
df['TechSupport']

[0 1]


0       0
1       0
2       0
3       1
4       0
       ..
7038    1
7039    0
7040    0
7041    0
7042    1
Name: TechSupport, Length: 7043, dtype: object

##### 8. Заполните пропуски в поле PhoneService значением 0

In [112]:
df['PhoneService'].fillna(0, inplace=True)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [117]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

new_df = pd.DataFrame()
for col in columns:
    new_df[col] = df[col]
df = new_df
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.85,0,0,0,0
1,Male,34,1,1889.5,0,0,0,0
2,Male,2,1,108.15,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [121]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.3,
                                                      random_state=21)

X_train

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport
1576,Female,68,1,3975.9,0,0,1
5013,Male,23,0,768.45,1,0,0
4077,Female,14,1,1346.3,1,1,0
3294,Female,1,1,69.1,0,0,0
3459,Female,30,1,789.55,0,0,0
...,...,...,...,...,...,...,...
6512,Male,11,1,688.5,0,0,1
48,Male,52,1,4217.8,1,1,1
772,Female,1,1,19.9,0,0,0
5944,Male,12,1,654.85,0,0,1


##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [196]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

In [233]:
class ToFloat(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.astype(float)

In [237]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender')),
                ('to_float', ToFloat())
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [238]:
gender.fit_transform(X_train, y_train)

,gender_Female,gender_Male
1576,1.0,0.0
5013,0.0,1.0
4077,1.0,0.0
3294,1.0,0.0
3459,1.0,0.0
...,...,...
6512,0.0,1.0
48,0.0,1.0
772,1.0,0.0
5944,0.0,1.0


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [239]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

__Ответ:__ Это полезно, когда мы обучаем линейную модель или метод главных компонентов. Это позволяет ускорить процесс обучения с тем же eta.

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [240]:
class NumberSelectorImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X2 = X[[self.key]]
        X2.loc[X2[self.key] == '', self.key] = 0.0
        return X2


TotalCharges = Pipeline([
    ('selector_imputer', NumberSelectorImputer(key='TotalCharges')),
    ('standard', StandardScaler())
])

Pandas почему-то ругается на пустые строки в признаке TotalCharges

In [241]:
# X_train.loc[X_train['TotalCharges'] == '', 'TotalCharges']
# X_train.loc[X_train['TotalCharges'].str.match(r'[a-z]'), 'TotalCharges']
# X_train.loc[X_train['TotalCharges'].str.len() == 0, 'TotalCharges']
# X_train['TotalCharges'].apply(lambda row: float(row))
# X_train['TotalCharges'].astype(np.float)

Но я ненахожу ни одну. А если так подумать, а не такой уж и нужный этот признак. Обучу без него

Объединение всех "кубиков" очень легко сделать таким образом

In [242]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport'])),
                ('to_float', ToFloat())
            ])

In [243]:
feats = FeatureUnion([('tenure', tenure),
                      #('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)
                     ])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

Sklearn ругается ValueError: Unknown label type: 'unknown'

In [192]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4930 entries, 1576 to 5327
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   gender           4930 non-null   object
 1   tenure           4930 non-null   int64 
 2   PhoneService     4930 non-null   int64 
 3   TotalCharges     4930 non-null   object
 4   StreamingMovies  4930 non-null   object
 5   StreamingTV      4930 non-null   object
 6   TechSupport      4930 non-null   object
dtypes: int64(2), object(5)
memory usage: 437.2+ KB


Все дело вот, что тип у них не числовой, а object

In [258]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('to_float', ToFloat())#,
    #('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tenure',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler(copy=True,
                                                                                 with_mean=True,
                                                                                 with_std=True))],
                                                          verbose=False)),
                                                ('continuos_features',
                                                 Pipeline(memory=None,
                  

In [259]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4930 entries, 1576 to 5327
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   gender           4930 non-null   object
 1   tenure           4930 non-null   int64 
 2   PhoneService     4930 non-null   int64 
 3   TotalCharges     4930 non-null   object
 4   StreamingMovies  4930 non-null   object
 5   StreamingTV      4930 non-null   object
 6   TechSupport      4930 non-null   object
dtypes: int64(2), object(5)
memory usage: 437.2+ KB


Опять object-ы. Пошло все к черту!

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [ ]:
#Ваш код здесь


##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss

#Ваш код здесь


### Сохраним наш пайплайн

In [ ]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)